In [ ]:
import serial
import struct
import datetime

In [ ]:
GET_SAMPLING_RATE_COMMAND = 0x03
SAMPLING_RATE_RESPONSE = 0x04
ACK_COMMAND_PROCESSED = 0xFF
STOP_STREAMING_COMMAND = 0x20
START_STREAMING_COMMAND = 0x07

In [ ]:
# For every packet that the Shimmer3 receives, it sends an acknowledgement message
# (ACK_COMMAND_PROCESSED) back to the host, to acknowledge receipt of the command.
def wait_for_ack():
    """Wait for acknowledgement bit from device.

    Keyword arguments:
    ser -- serial port
    """
    ddata = ""
    ack = struct.pack('B', ACK_COMMAND_PROCESSED) # pack the given Python values according to the given format. 'B' means unsigned byte, the value passed to struct.pack() should be interpreted as an unsigned byte (8bits)
    # it packs the hexadeicmal value '0xff' into an unsigned btyes.
    while ddata != ack:
        ddata = ser.read(1)
        print(("received [%s]" % struct.unpack('B', ddata)))
        return

In [ ]:
def accel_calc(data):
    """Calculate m/s^2 values of accelerometer from raw output.

    Keyword arguments:
    data -- raw data from device
    """
    value = (data - 2047) * float(1) / float(830) * float(9.80665)
    return value

def gyro_calc(data):
    """Calculate deg/s values of gyroscope from raw output.

    Keyword arguments:
    data -- raw data from device
    """
    value = data / float(65.5)
    return value

def mag_calc_xy(data):
    """Calculate gauss values of magnometer from raw output, X and Y.

    Keyword arguments:
    data -- raw data from device
    """
    value = data / float(1100)
    return value

def mag_calc_z(data):
    """Calculate gauss values of magnometer from raw output, Z.

    Keyword arguments:
    data -- raw data from device
    """
    value = data / float(980)
    return value

def gsr_calc(data):
    """Calculate gsr value.

    Keyword arguments:
    data -- raw data from device
    """
    if (gsr.gsrrange == 0):
        p1 = 0.0363
        p2 = -24.8617
    if (gsr.gsrrange == 1):
        p1 = 0.0051
        p2 = -3.8357
    if (gsr.gsrrange == 2):
        p1 = 0.0015
        p2 = -1.0067
    if (gsr.gsrrange == 3):
        p1 = 4.4513e-04
        p2 = -0.3193
    value = (float(p1) * data) + float(p2)
    return value


def ppg_calc(data):
    """PLACEHOLDER."""
    value = data * float(0.00024414062)
    return value

In [ ]:
# Open Serial Port
ser = serial.Serial('/dev/rfcomm3', 115200)
ser.flushInput()

In [ ]:
# Stop Streaming
ser.write(struct.pack('B', STOP_STREAMING_COMMAND))
wait_for_ack()

In [ ]:
# Get Sampling Rate
# when the Shimmer3 receives the GET_SAMPLING_RATE_COMMAND, it must send the
# current value of the sampling rate back to the host; 
# the response packet will contain two bytes: the first byte will be the SAMPLING_RATE_REPONSE identifier
# and the second byte will be the sampling
# rate value. 
ser.write(struct.pack('B', GET_SAMPLING_RATE_COMMAND))
wait_for_ack()
ddata = ser.read(3)
ddata = struct.unpack('<BH', ddata)
print('Sampling Rate is: %f' % (32768 / ddata[1]))

In [ ]:
# Start Streaming
ser.write(struct.pack('B', 0x07))
wait_for_ack()
numbytes = 0
ddata = b''
framesize = 1 + 3 + 2*3 + 2*3 + 2*3 + 7
while True:
    while numbytes < framesize:
        ddata += ser.read(framesize)
        numbytes = len(ddata)

    data = ddata[0:framesize]
    numbytes = 0
    ddata = b''

    (packettype) = struct.unpack('B', data[0:1]) #[0]

    (lin_acc_raw_x, lin_acc_raw_y, lin_acc_raw_z) = struct.unpack('HHH', data[4:10])
    lin_acc_cal_x = accel_calc(lin_acc_raw_x)
    lin_acc_cal_y = accel_calc(lin_acc_raw_y)
    lin_acc_cal_z = accel_calc(lin_acc_raw_z)
    
    (ang_vel_raw_x, ang_vel_raw_y, ang_vel_raw_z) = struct.unpack('>hhh', data[10:16])
    ang_vel_cal_x = gyro_calc(ang_vel_raw_x)
    ang_vel_cal_y = gyro_calc(ang_vel_raw_y)
    ang_vel_cal_z = gyro_calc(ang_vel_raw_z)

    (mag_fld_raw_x, mag_fld_raw_z, mag_fld_raw_y) = struct.unpack('>hhh', data[16:22])
    mag_fld_cal_x = mag_calc_xy(mag_fld_raw_x)
    mag_fld_cal_y = mag_calc_xy(mag_fld_raw_y)
    mag_fld_cal_z = mag_calc_z(mag_fld_raw_z)

    emg_ch1_raw = struct.unpack('>i', (data[23:26] + b'\0'))[0] >> 8
    emg_ch2_raw = struct.unpack('>i', (data[26:29] + b'\0'))[0] >> 8

    emg_ch1_cal = emg_calc(emg_ch1_raw)
    emg_ch2_cal = emg_calc(emg_ch2_raw)  

    print(lin_acc_cal_x)
    

In [ ]:
#framesize = 1 + 3 + 3 * 2

#prev_time = datetime.datetime.now()
#while True:
#    ddata = b''
#    numbytes = 0
#    read = True
#    while read == True:
#        flag = ser.read(1)
#        flag = struct.unpack('B', flag)
#        if flag[0] == 0:
#            ddata += ser.read(framesize-1)
##            read = False
#    current_time = datetime.datetime.now()
#    print(current_time - prev_time)
#    prev_time = current_time
    # (accel_x, accel_y, accel_z) = struct.unpack('HHH', ddata[3:9])
    # accel_y_cal = accel_calc(accel_y)
    # print(accel_y_cal)
